Goal is to predict the next time series within the sequence

In [2]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input, LSTM, Dropout
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner import RandomSearch
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from matplotlib import pyplot as plt
from IPython import display
from ann_visualizer.visualize import ann_viz 

In [3]:
df = pd.read_csv("../datasets/LSTMdataset.csv",index_col=0)
df["DateTime"] = pd.to_datetime(df["DateTime"], format = "%Y%m%d %H:%M:%S")
df

,DateTime,Holiday,Temperature,Snow_Depth,Wind_Speed,Clear,Cloudy,Overcast,Rain
2,2001-01-03 15:00:00,0,22.5,6.77,10.8,0,1,0,0
9,2001-01-10 22:15:00,0,27.6,3.72,20.4,0,1,0,0
10,2001-01-11 18:30:00,0,35.5,3.72,14.6,0,1,0,0
11,2001-01-12 19:30:00,0,34.2,3.15,8.7,1,0,0,0
16,2001-01-17 09:40:00,0,37.9,0.00,13.9,0,0,1,0
...,...,...,...,...,...,...,...,...,...
9051,2021-11-07 03:56:32,0,39.4,0.00,6.8,1,0,0,0
9054,2021-11-10 08:13:41,0,51.8,0.00,15.4,1,0,0,0
9057,2021-11-13 14:56:29,0,41.9,0.00,15.6,0,0,0,1
9060,2021-11-16 20:14:30,0,38.2,0.00,7.3,1,0,0,0


https://machinelearningmastery.com/how-to-use-the-timeseriesgenerator-for-time-series-forecasting-in-keras/

In [4]:
# Split training and test data (80% split)
train_index = round(len(df) * .8)
train_data = df[:train_index]
test_data = df[train_index:]
scaler = MinMaxScaler() # This is used convert the values to 0-1 for model to learn quicker
scaler.fit(train_data)
scaled_train = scaler.transform(train_data)
scaled_test = scaler.transform(test_data)


n_input = 3 #length of inputs to predict the output
generator = TimeseriesGenerator(train_data,targets=train_data,length= n_input, batch_size = 1)

TypeError: The DType <class 'numpy.dtype[datetime64]'> could not be promoted by <class 'numpy.dtype[float64]'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtype[datetime64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[float64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>, <class 'numpy.dtype[int64]'>)

In [43]:
def build_model(hp):
    model = Sequential()

    # Finds how many layers should be in NN
    for layer in range(hp.Int("num_layers", 1, 5)):
        model.add(LSTM(units=hp.Int("units_" + str(layer), min_value=100, max_value=250, step=25), input_shape=(9,1), activation="relu", return_sequences=True))  # Find best amount of neurons for layer
        model.add(Dropout(.2))
    
    model.add(Dense(1))
    model.compile(optimizer=Adam(hp.Choice('learning_rate', values=[1e-2, 1e-4])), loss='mse', metrics=["mean_squared_error"])  # Adam = SGD
    return model

In [44]:
# Stop training when loss doesn't imporve in 1 epoches
callback = EarlyStopping(monitor="loss", min_delta=.01, patience=1)
tuner = RandomSearch(
    build_model,
    objective='mse',
    directory='LSTM_trails',  # Store results
    project_name='LSTM_model',
    overwrite=True, 
    max_trials=12,  # Bc 6 steps x 2 learning_rates
    executions_per_trial=3  # How many times to train verison of model
)
tuner.search(generator, callbacks=[callback], validation_split=.5)


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
2                 |?                 |num_layers
225               |?                 |units_0
0.01              |?                 |learning_rate



ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type Timestamp).